In [ ]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from GTSRBNet import GTSRBNet
from GTSRBDataset import GTSRBDataset
from matplotlib import pyplot as plt

In [ ]:
root = ''
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0))])

validset = GTSRBDataset('valid_us.npz', transform=transform)
validloader = torch.utils.data.DataLoader(validset, batch_size=256,
                                          shuffle=True, num_workers=2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = GTSRBNet()
model.to(device)

classes = []
with open(root + 'class_semantics.txt') as f:
    for line in f:
        classes.append(line.strip())

checkpoint = torch.load('checkpoint_us.tar')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

In [ ]:
ite = next(iter(validloader)) #batch of 256
classidx = ite[1][0]
o_class = classes[classidx]
print(o_class)
print(classidx)
classes[16]
input_ime = ite[0][0].cuda()
input_im = input_ime.unsqueeze(0)
imshow(input_ime.cpu())

In [ ]:
#TODO: Bound the ft/gy so that the image is a valid rgb image.
#TODO: See if Conv1d is an accurate representation of the convolution

# The input time signal
ft = torch.ones([1,1,36,1], requires_grad=True, dtype=torch.float, device=device)

#The shutter function is encoded into the convolution layer
lay = torch.nn.Conv1d(1,1,5)
lay.to(device)

#Manually setting the weights and bias so the  shutter acts as a box filter
lay.weight.data = torch.ones([1,1,5,1], requires_grad=True, dtype=torch.float, device=device)
lay.bias.data = torch.zeros(1, requires_grad=True, dtype=torch.float, device=device)

#Target and original class labels
target = torch.tensor([16], dtype=torch.long, device=device)
orig = torch.tensor([classidx], dtype=torch.long, device=device)
targloss = []
origloss = []

lr = 1e-1
n_epochs = 1000
optimizer = optim.SGD([ft], lr=lr)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(n_epochs):
    
    # Compute g(y) to get X_adv
    gy = lay(ft)    #Convolution of ft and the shutter
    inp = input_im * gy           #gy is broadcasted to match the shape of input_im
    out = model(inp)
    
    #Calculate Loss
    loss = loss_fn(out, target)
    targloss.append(loss.data)
    origloss.append(loss_fn(out,orig))
    loss.backward()
    
    optimizer.step()
    
    optimizer.zero_grad()
    
plt.plot(targloss, label="target")
plt.plot(origloss, label="original")
plt.legend()
plt.show()

In [ ]:
plt.plot(targloss[100:], label="target")
plt.plot(origloss[100:], label="original")
plt.legend()
plt.show()

In [ ]:
def imshow(img):
    img = img + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
inp2 = (input_ime - torch.min(input_ime))
inp2 /= torch.max(inp2)
imshow(inp2.cpu())
imshow(input_ime.cpu())

In [ ]:
inp2 = (inp - torch.min(inp))
inp2 /= torch.max(inp2)
imshow(inp2[0].detach().cpu())
imshow(inp[0].detach().cpu())

In [ ]:
test = torch.ones([1,3,32,32], device=device)
test2 = gy - torch.min(gy)
test2 /= torch.max(test2)
test = test * test2
imshow(test[0].detach().cpu())

In [ ]:
plt.plot(torch.flatten(ft).detach().cpu())
plt.plot(torch.flatten(gy).detach().cpu())
plt.show()
plt.plot(torch.flatten(test2).detach().cpu())
plt.show()

In [ ]:
gy

In [ ]:
print(torch.min(input_im),torch.max(input_im))